# simulationResults.

This notebook takes simulates flux profiles and lipid yields for the *L. starkeyi* GSM. 

import needed libraries.

In [41]:
import cobra
import pandas
import numpy
import re
from cobra.flux_analysis import pfba


load GSM models.

In [42]:
# Lipomyces starkeyi constructed model (based on NRRL 11557 genome and 11558 phenotypic data).
lst = cobra.io.load_json_model('../models/Lst_v0_4_forPUB.json')

# Rhodosporidium toruloides IFOxxxx.
rto = cobra.io.load_json_model('../models/Rt_IFO0880.json')

# Yarrowia lipolytica POxx.
ylip = cobra.io.load_matlab_model('../models/twoModels/iYLI647_corr.mat')

# Sacchromyces cerevisiae Yeast8.
sce = cobra.io.load_matlab_model('../models/yeast8_modifiedwBIGGnames.mat')


# E. coli iJO1366
eco = cobra.io.load_json_model('../models/iJO1366.json')

This model seems to have confidenceScores instead of rxnConfidenceScores field. Will use confidenceScores for what rxnConfidenceScores represents.
This model seems to have metCharge instead of metCharges field. Will use metCharge for what metCharges represents.
No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, m, n, r, v, x


In [3]:
# adjust iYLI647 parameters for consistent annotation. 
for m in ylip.metabolites:
    if ('[' in m.id):
        ylip.metabolites.get_by_id(m.id).id = m.id.replace('[','_').replace(']','')
    if ('_L_' in m.id):
        ylip.metabolites.get_by_id(m.id).id = m.id.replace('_L_','__L_')
    if ('_D_' in m.id):
        ylip.metabolites.get_by_id(m.id).id = m.id.replace('_D_','__D_')
for r in ylip.reactions:
    if ('(e)' in r.id):
        ylip.reactions.get_by_id(r.id).id = r.id.replace('(e)','_e')
    if(('_L_') in r.id):
        ylip.reactions.get_by_id(r.id).id = r.id.replace('_L_','__L_')

    if(('_D_') in r.id):
        ylip.reactions.get_by_id(r.id).id = r.id.replace('_D_','__D_')


view summary of the GSMs.

In [43]:
lst.reactions.EX_glc__D_e.lower_bound=-1
# lst.reactions.EX_xyl__D_e.upper_bound=0

lst.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,5.053E-05,0,0.00%
cu2_e,EX_cu2_e,3.189E-05,0,0.00%
fe3_e,EX_fe3_e,0.0003747,0,0.00%
glc__D_e,EX_glc__D_e,1,6,98.03%
k_e,EX_k_e,0.03557,0,0.00%
mg2_e,EX_mg2_e,0.003748,0,0.00%
mn2_e,EX_mn2_e,3.687E-05,0,0.00%
na1_e,EX_na1_e,0.001585,0,0.00%
nh4_e,EX_nh4_e,0.5131,0,0.00%
o2_e,EX_o2_e,1.799,0,0.00%


In [5]:
for rx in lst.reactions:
    print(rx)

ALCD25yi: h_c + nadph_c + pacald_c --> 2phetoh_c + nadp_c
MTHFCm: h2o_m + methf_m <=> 10fthf_m + h_m
AMPN: amp_c + h2o_c --> ade_c + r5p_c
DAGCPTer_LS: 0.01 12dgr_LS_r + cdpchol_r --> cmp_r + h_r + 0.01 pc_LS_r
PYRt2: h_e + pyr_e --> h_c + pyr_c
NNDPRm: 2.0 h_m + prpp_m + quln_m --> co2_m + nicrnt_m + ppi_m
HMGCOASm: coa_m + h_m + hmgcoa_m <=> aacoa_m + accoa_m + h2o_m
PDE4: 35cgmp_c + h2o_c --> gmp_c + h_c
PAPSR: paps_c + trdrd_c --> 2.0 h_c + pap_c + so3_c + trdox_c
ACOAO4p: dcacoa_x + o2_x --> dc2coa_x + h2o2_x
3IPM3MT: 3c2hmp_c + amet_c --> 3ipmmest_c + ahcys_c
UDPGD: h2o_c + 2.0 nad_c + udpg_c --> 3.0 h_c + 2.0 nadh_c + udpglcur_c
CYTDK2: cytd_c + gtp_c --> cmp_c + gdp_c + h_c
FACOAL141: atp_c + coa_c + ttdcea_c --> amp_c + ppi_c + tdecoa_c
ACACT7p: 3ohdcoa_x + coa_x --> accoa_x + tdcoa_x
POLYAO: N1aspmd_c + h2o_c + o2_c --> 3aap_c + h2o2_c + ptrc_c
MEVK1: atp_c + mev__R_c --> 5pmev_c + adp_c + h_c
DATUP: datp_c + uri_c --> dadp_c + h_c + ump_c
TAUDO: akg_c + o2_c + taur_c --> aac

In [6]:
rto.reactions.EX_glc__D_e.lower_bound=-1

rto.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,6.104E-05,0,0.00%
cu2_e,EX_cu2_e,3.852E-05,0,0.00%
fe3_e,EX_fe3_e,0.0004526,0,0.00%
glc__D_e,EX_glc__D_e,1,6,98.64%
k_e,EX_k_e,0.04297,0,0.00%
mg2_e,EX_mg2_e,0.004528,0,0.00%
mn2_e,EX_mn2_e,4.453E-05,0,0.00%
na1_e,EX_na1_e,0.001915,0,0.00%
nh4_e,EX_nh4_e,0.4842,0,0.00%
o2_e,EX_o2_e,2.287,0,0.00%


In [7]:
ylip.reactions.EX_glc_e.lower_bound=-1
ylip.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc_e,1,6,100.00%
h_e,EX_h_e,0.03853,0,0.00%
nh4_e,EX_nh4_e,0.7772,0,0.00%
o2_e,EX_o2_e,1.499,0,0.00%
pi_e,EX_pi_e,0.03853,0,0.00%
so4_e,EX_so4_e,0.007735,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
biomass_c,EX_biom,-0.114,0,0.00%
co2_e,EX_co2_e,-1.679,1,100.00%
h2o_e,EX_h2o_e,-3.253,0,0.00%


In [8]:
sce.reactions.EX_glc__D_e.lower_bound=-1
sce.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,1.778E-05,0,0.00%
cl_e,EX_cl_e,0.0001057,0,0.00%
cu2_e,EX_cu2_e,5.399E-05,0,0.00%
fe2_e,EX_fe2_e,2.573E-06,0,0.00%
glc__D_e,EX_glc__D_e,1,6,100.00%
h_e,EX_h_e,0.04144,0,0.00%
k_e,EX_k_e,0.0002974,0,0.00%
mg2_e,EX_mg2_e,0.0001018,0,0.00%
mn2_e,EX_mn2_e,0.0002237,0,0.00%
na1_e,EX_na1_e,0.0003253,0,0.00%


In [9]:
eco.reactions.EX_glc__D_e.lower_bound=-1
eco.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0004501,0,0.00%
cl_e,EX_cl_e,0.0004501,0,0.00%
cobalt2_e,EX_cobalt2_e,2.162E-06,0,0.00%
cu2_e,EX_cu2_e,6.131E-05,0,0.00%
fe2_e,EX_fe2_e,0.001389,0,0.00%
glc__D_e,EX_glc__D_e,1,6,100.00%
k_e,EX_k_e,0.01688,0,0.00%
mg2_e,EX_mg2_e,0.0007502,0,0.00%
mn2_e,EX_mn2_e,5.976E-05,0,0.00%
mobd_e,EX_mobd_e,1.116E-05,0,0.00%


growth rates of the species on 1 mmol/gDCW/h of D-glucose: 


|               | *S. cerevisiae*|*R. toruloides*|*Y. lipolytica*|*L. starkeyi*|
|---------------|---------------|---------------|---------------|-------------|
|mu (h<sup>-1</sup>)       | 0.082         | 0.073         | 0.114         | 0.089       |
|CO2 uptake $\frac{mmol}{gDCW*h}$    | -2.468        |  -2.561       | -1.679        | -2.253      |
|O2  uptake $\frac{mmol}{gDCW*h}$     | 2.332         |2.287	        | 1.499         | 1.799       |
|HCO3<sup>-</sup> uptake $\frac{mmol}{gDCW*h}$   | -             | -0.001839     | -             | -0.002477   |
|RQ             | 1.058         | 1.121         | 1.120         |1.253|

In [12]:
2.468/2.332

1.058319039451115

In [13]:
(2.561+0.002)/2.287

1.1206821163095757

In [14]:
1.679/1.499

1.1200800533689126

In [15]:
(2.253+0.002477)/1.799

1.253739299610895

In [16]:

def count_carbons_in_formula(formula):
    """
    Count the number of carbons in a given chemical formula.
    """
    match = re.search(r"C(\d+)", formula)
    if match:
        return int(match.group(1))
    return 0

#### compare simulated flux on different carbon sources.

In [17]:
lst_glc_soln = lst.optimize()
rto_glc_soln = rto.optimize()
ylip_glc_soln = ylip.optimize()
sce_glc_soln = sce.optimize()
eco_glc_soln = eco.optimize()


In [18]:
soln = pfba(lst)

In [19]:
soln.fluxes.to_csv('pfba_glc_for_fig_FORPUB.csv')

#### Max lipid yield from glucose. 

In [20]:
with lst:
    
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f} growth on carbon source')
    
    
    lst.objective = 'SK_triglyc_LS_d'
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f}')
    print(lst.objective)
    
    carbon_source_num_carbons = count_carbons_in_formula(lst.metabolites.glc__D_e.formula)
    tag_num_carbons = count_carbons_in_formula(lst.metabolites.triglyc_LS_d.formula)
    
    
    mgUptake = 1*lst.metabolites.glc__D_e.formula_weight
    tag_produced = soln.objective_value*lst.metabolites.triglyc_LS_d.formula_weight
    
    print(f'the yield from 1 mmol of glucose is {tag_produced/mgUptake: .2f} g TAG/g carbon consumed')
    print(f'the yield from 1 mmol of glucose is {soln.objective_value*tag_num_carbons/carbon_source_num_carbons: .2f} mmol-C TAG/mmol-C carbon consumed')

    
    
    print(f'{tag_produced:.2f} mg of TAG')
    print(f'{mgUptake:.2f} mg of carbon consumed')

0.0889 growth on carbon source
0.0007
Maximize
1.0*SK_triglyc_LS_d - 1.0*SK_triglyc_LS_d_reverse_50347
the yield from 1 mmol of glucose is  0.33 g TAG/g carbon consumed
the yield from 1 mmol of glucose is  0.63 mmol-C TAG/mmol-C carbon consumed
59.32 mg of TAG
180.16 mg of carbon consumed


In [21]:
# yield on xylose

with lst:
    lst.reactions.EX_glc__D_e.lower_bound = 0
    lst.reactions.EX_xyl__D_e.lower_bound = -1
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f} growth on carbon source')
    
    
    lst.objective = 'SK_triglyc_LS_d'
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f}')
    print(lst.objective)
    
    carbon_source_num_carbons = count_carbons_in_formula(lst.metabolites.xyl__D_e.formula)
    tag_num_carbons = count_carbons_in_formula(lst.metabolites.triglyc_LS_d.formula)
    
    mgUptake = 1*lst.metabolites.xyl__D_e.formula_weight
    tag_produced = soln.objective_value*lst.metabolites.triglyc_LS_d.formula_weight
    print(f'the yield from 1 mmol of carbon is {tag_produced/mgUptake:.2f} g TAG/carbon consumed')        
    print(f'the yield from 1 mmol of carbon is {soln.objective_value*tag_num_carbons/carbon_source_num_carbons: .2f} mmol-C TAG/mmol-C carbon consumed')

    
    
    print(f'{tag_produced:2f} mg of TAG')
    print(f'{mgUptake:.2f} mg of carbon consumed')

0.0731 growth on carbon source
0.0006
Maximize
1.0*SK_triglyc_LS_d - 1.0*SK_triglyc_LS_d_reverse_50347
the yield from 1 mmol of carbon is 0.32 g TAG/carbon consumed
the yield from 1 mmol of carbon is  0.61 mmol-C TAG/mmol-C carbon consumed
47.934002 mg of TAG
150.13 mg of carbon consumed


In [22]:
# yield on sucrose

with lst:
    lst.reactions.EX_glc__D_e.lower_bound = 0
    lst.reactions.EX_sucr_e.lower_bound = -1
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f} growth on carbon source')
    
    lst.objective = 'SK_triglyc_LS_d'

    soln = lst.optimize()
    print(f'{soln.objective_value:.4f}')
    print(lst.objective)
    
    carbon_source_num_carbons = count_carbons_in_formula(lst.metabolites.sucr_e.formula)
    tag_num_carbons = count_carbons_in_formula(lst.metabolites.triglyc_LS_d.formula)
    
    mgUptake = 1*lst.metabolites.sucr_e.formula_weight
    tag_produced = soln.objective_value*lst.metabolites.triglyc_LS_d.formula_weight
    
    print(f'the yield from 1 mmol of carbon is {tag_produced/mgUptake: .2f} g TAG/carbon consumed')
    print(f'the yield from 1 mmol of carbon is {soln.objective_value*tag_num_carbons/carbon_source_num_carbons: .2f} mmol-C TAG/mmol-C carbon consumed')

    
    print(f'{tag_produced:.2f} mg of TAG')
    print(f'{mgUptake:.2f} mg of carbon consumed')

0.1786 growth on carbon source
0.0014
Maximize
1.0*SK_triglyc_LS_d - 1.0*SK_triglyc_LS_d_reverse_50347
the yield from 1 mmol of carbon is  0.35 g TAG/carbon consumed
the yield from 1 mmol of carbon is  0.63 mmol-C TAG/mmol-C carbon consumed
119.13 mg of TAG
342.30 mg of carbon consumed


In [23]:
# yield on glycerol

with lst:
    lst.reactions.EX_glc__D_e.lower_bound = 0
    lst.reactions.EX_glyc_e.lower_bound = -1
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f} growth on carbon source')
    
    lst.objective = 'SK_triglyc_LS_d'

    soln = lst.optimize()
    print(f'{soln.objective_value:.4f}')
    print(lst.objective)
    
    carbon_source_num_carbons = count_carbons_in_formula(lst.metabolites.glyc_e.formula)
    tag_num_carbons = count_carbons_in_formula(lst.metabolites.triglyc_LS_d.formula)    
    
    mgUptake = 1*lst.metabolites.glyc_e.formula_weight
    tag_produced = soln.objective_value*lst.metabolites.triglyc_LS_d.formula_weight
    
    print(f'the yield from 1 mmol of carbon is {tag_produced/mgUptake: .2f} g TAG/carbon consumed')
    print(f'the yield from 1 mmol of carbon is {soln.objective_value*tag_num_carbons/carbon_source_num_carbons: .2f} mmol-C TAG/mmol-C carbon consumed')

    print(f'{tag_produced:.2f} mg of TAG')
    print(f'{mgUptake:.2f} mg of carbon consumed')

0.0511 growth on carbon source
0.0004
Maximize
1.0*SK_triglyc_LS_d - 1.0*SK_triglyc_LS_d_reverse_50347
the yield from 1 mmol of carbon is  0.37 g TAG/carbon consumed
the yield from 1 mmol of carbon is  0.72 mmol-C TAG/mmol-C carbon consumed
33.90 mg of TAG
92.09 mg of carbon consumed


In [24]:
# yield on gluconate

with lst:
    lst.reactions.EX_glc__D_e.lower_bound = 0
    lst.reactions.EX_glcn_e.lower_bound = -1
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f} growth on carbon source')
    
    lst.objective = 'SK_triglyc_LS_d'

    soln = lst.optimize()
    print(f'{soln.objective_value:.4f}')
    print(lst.objective)
    
    carbon_source_num_carbons = count_carbons_in_formula(lst.metabolites.glcn_e.formula)
    tag_num_carbons = count_carbons_in_formula(lst.metabolites.triglyc_LS_d.formula)
    
    mgUptake = 1*lst.metabolites.glcn_e.formula_weight
    tag_produced = soln.objective_value*lst.metabolites.triglyc_LS_d.formula_weight
    
    print(f'the yield from 1 mmol of carbon is {tag_produced/mgUptake: .2f} g TAG/carbon consumed')
    print(f'the yield from 1 mmol of carbon is {soln.objective_value*tag_num_carbons/carbon_source_num_carbons: .2f} mmol-C TAG/mmol-C carbon consumed')
    
    print(f'{tag_produced:.2f} mg of TAG')
    print(f'{mgUptake:.2f} mg of carbon consumed')

0.0814 growth on carbon source
0.0006
Maximize
1.0*SK_triglyc_LS_d - 1.0*SK_triglyc_LS_d_reverse_50347
the yield from 1 mmol of carbon is  0.28 g TAG/carbon consumed
the yield from 1 mmol of carbon is  0.58 mmol-C TAG/mmol-C carbon consumed
54.24 mg of TAG
195.15 mg of carbon consumed


In [25]:
# yield on Hexadecanoate

with lst:
    lst.reactions.EX_glc__D_e.lower_bound = 0
    lst.reactions.EX_hdca_e.lower_bound = -1
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f} growth on carbon source')

    
    lst.objective = 'SK_triglyc_LS_d'

    soln = lst.optimize()
    print(f'{soln.objective_value:.4f}')
    print(lst.objective)
    
    carbon_source_num_carbons = count_carbons_in_formula(lst.metabolites.hdca_e.formula)
    tag_num_carbons = count_carbons_in_formula(lst.metabolites.triglyc_LS_d.formula)
    
    mgUptake = 1*lst.metabolites.hdca_e.formula_weight
    tag_produced = soln.objective_value*lst.metabolites.triglyc_LS_d.formula_weight
    
    print(f'the yield from 1 mmol of carbon is {tag_produced/mgUptake: .2f} g TAG/carbon consumed')
    print(f'the yield from 1 mmol of carbon is {soln.objective_value*tag_num_carbons/carbon_source_num_carbons: .2f} mmol-C TAG/mmol-C carbon consumed')
    
    print(f'{tag_produced:.2f} mg of TAG')
    print(f'{mgUptake:.2f} mg of carbon consumed')

0.2691 growth on carbon source
0.0027
Maximize
1.0*SK_triglyc_LS_d - 1.0*SK_triglyc_LS_d_reverse_50347
the yield from 1 mmol of carbon is  0.92 g TAG/carbon consumed
the yield from 1 mmol of carbon is  0.94 mmol-C TAG/mmol-C carbon consumed
234.49 mg of TAG
255.42 mg of carbon consumed


In [26]:
# yield on maltose.

with lst:
    lst.reactions.EX_glc__D_e.lower_bound = 0
    lst.reactions.EX_malt_e.lower_bound = -1
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f} growth on carbon source')
    
    lst.objective = 'SK_triglyc_LS_d'

    soln = lst.optimize()
    print(f'{soln.objective_value:.4f}')
    print(lst.objective)
    
    carbon_source_num_carbons = count_carbons_in_formula(lst.metabolites.malt_e.formula)
    tag_num_carbons = count_carbons_in_formula(lst.metabolites.triglyc_LS_d.formula)
    
    mgUptake = 1*lst.metabolites.malt_e.formula_weight
    tag_produced = soln.objective_value*lst.metabolites.triglyc_LS_d.formula_weight
    
    print(f'the yield from 1 mmol of carbon is {tag_produced/mgUptake: .2f} g TAG/carbon consumed')
    print(f'the yield from 1 mmol of carbon is {soln.objective_value*tag_num_carbons/carbon_source_num_carbons: .2f} mmol-C TAG/mmol-C carbon consumed')
    
    print(f'{tag_produced:.2f} mg of TAG')
    print(f'{mgUptake:.2f} mg of carbon consumed')

0.1786 growth on carbon source
0.0014
Maximize
1.0*SK_triglyc_LS_d - 1.0*SK_triglyc_LS_d_reverse_50347
the yield from 1 mmol of carbon is  0.35 g TAG/carbon consumed
the yield from 1 mmol of carbon is  0.63 mmol-C TAG/mmol-C carbon consumed
119.13 mg of TAG
342.30 mg of carbon consumed


In [27]:
# yield on acetate

with lst:
    lst.reactions.EX_glc__D_e.lower_bound = 0
    lst.reactions.EX_ac_e.lower_bound = -1
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f} growth on carbon source')
    
    lst.objective = 'SK_triglyc_LS_d'

    soln = lst.optimize()
    print(f'{soln.objective_value:.4f}')
    print(lst.objective)
    
    carbon_source_num_carbons = count_carbons_in_formula(lst.metabolites.ac_e.formula)
    tag_num_carbons = count_carbons_in_formula(lst.metabolites.triglyc_LS_d.formula)
    
    mgUptake = 1*lst.metabolites.ac_e.formula_weight
    tag_produced = soln.objective_value*lst.metabolites.triglyc_LS_d.formula_weight
    
    print(f'the yield from 1 mmol of carbon is {tag_produced/mgUptake: .2f} g TAG/carbon consumed')
    print(f'the yield from 1 mmol of carbon is {soln.objective_value*tag_num_carbons/carbon_source_num_carbons: .2f} mmol-C TAG/mmol-C carbon consumed')
    
    print(f'{tag_produced:.2f} mg of TAG')
    print(f'{mgUptake:.2f} mg of carbon consumed')

0.0227 growth on carbon source
0.0002
Maximize
1.0*SK_triglyc_LS_d - 1.0*SK_triglyc_LS_d_reverse_50347
the yield from 1 mmol of carbon is  0.27 g TAG/carbon consumed
the yield from 1 mmol of carbon is  0.52 mmol-C TAG/mmol-C carbon consumed
16.19 mg of TAG
59.04 mg of carbon consumed


### Look at yield with 10% confined growth

In [28]:
percent = 0.1

In [29]:
with lst:
    
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f} growth on carbon source')
    
    
    lst.reactions.BIOMASS_Ls.lower_bound = percent*soln.objective_value
    lst.objective = 'SK_triglyc_LS_d'
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f}')
    print(lst.objective)
    
    carbon_source_num_carbons = count_carbons_in_formula(lst.metabolites.glc__D_e.formula)
    tag_num_carbons = count_carbons_in_formula(lst.metabolites.triglyc_LS_d.formula)
    
    
    mgUptake = 1*lst.metabolites.glc__D_e.formula_weight
    tag_produced = soln.objective_value*lst.metabolites.triglyc_LS_d.formula_weight
    
    print(f'the yield from 1 mmol of glucose is {tag_produced/mgUptake: .2f} g TAG/g carbon consumed')
    print(f'the yield from 1 mmol of glucose is {soln.objective_value*tag_num_carbons/carbon_source_num_carbons: .2f} mmol-C TAG/mmol-C carbon consumed')

    
    
    print(f'{tag_produced:.2f} mg of TAG')
    print(f'{mgUptake:.2f} mg of carbon consumed')

0.0889 growth on carbon source
0.0006
Maximize
1.0*SK_triglyc_LS_d - 1.0*SK_triglyc_LS_d_reverse_50347
the yield from 1 mmol of glucose is  0.30 g TAG/g carbon consumed
the yield from 1 mmol of glucose is  0.57 mmol-C TAG/mmol-C carbon consumed
53.59 mg of TAG
180.16 mg of carbon consumed


In [30]:
# yield on xylose

with lst:
    lst.reactions.EX_glc__D_e.lower_bound = 0
    lst.reactions.EX_xyl__D_e.lower_bound = -1
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f} growth on carbon source')
    
    lst.reactions.BIOMASS_Ls.lower_bound = percent*soln.objective_value
    lst.objective = 'SK_triglyc_LS_d'
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f}')
    print(lst.objective)
    
    carbon_source_num_carbons = count_carbons_in_formula(lst.metabolites.xyl__D_e.formula)
    tag_num_carbons = count_carbons_in_formula(lst.metabolites.triglyc_LS_d.formula)
    
    mgUptake = 1*lst.metabolites.xyl__D_e.formula_weight
    tag_produced = soln.objective_value*lst.metabolites.triglyc_LS_d.formula_weight
    print(f'the yield from 1 mmol of carbon is {tag_produced/mgUptake:.2f} g TAG/carbon consumed')        
    print(f'the yield from 1 mmol of carbon is {soln.objective_value*tag_num_carbons/carbon_source_num_carbons: .2f} mmol-C TAG/mmol-C carbon consumed')

    
    
    print(f'{tag_produced:2f} mg of TAG')
    print(f'{mgUptake:.2f} mg of carbon consumed')

0.0731 growth on carbon source
0.0005
Maximize
1.0*SK_triglyc_LS_d - 1.0*SK_triglyc_LS_d_reverse_50347
the yield from 1 mmol of carbon is 0.29 g TAG/carbon consumed
the yield from 1 mmol of carbon is  0.55 mmol-C TAG/mmol-C carbon consumed
43.219863 mg of TAG
150.13 mg of carbon consumed


In [31]:
# yield on sucrose

with lst:
    lst.reactions.EX_glc__D_e.lower_bound = 0
    lst.reactions.EX_sucr_e.lower_bound = -1
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f} growth on carbon source')
    lst.reactions.BIOMASS_Ls.lower_bound = percent*soln.objective_value

    
    lst.objective = 'SK_triglyc_LS_d'

    soln = lst.optimize()
    print(f'{soln.objective_value:.4f}')
    print(lst.objective)
    
    carbon_source_num_carbons = count_carbons_in_formula(lst.metabolites.sucr_e.formula)
    tag_num_carbons = count_carbons_in_formula(lst.metabolites.triglyc_LS_d.formula)
    
    mgUptake = 1*lst.metabolites.sucr_e.formula_weight
    tag_produced = soln.objective_value*lst.metabolites.triglyc_LS_d.formula_weight
    
    print(f'the yield from 1 mmol of carbon is {tag_produced/mgUptake: .2f} g TAG/carbon consumed')
    print(f'the yield from 1 mmol of carbon is {soln.objective_value*tag_num_carbons/carbon_source_num_carbons: .2f} mmol-C TAG/mmol-C carbon consumed')

    
    print(f'{tag_produced:.2f} mg of TAG')
    print(f'{mgUptake:.2f} mg of carbon consumed')

0.1786 growth on carbon source
0.0013
Maximize
1.0*SK_triglyc_LS_d - 1.0*SK_triglyc_LS_d_reverse_50347
the yield from 1 mmol of carbon is  0.31 g TAG/carbon consumed
the yield from 1 mmol of carbon is  0.57 mmol-C TAG/mmol-C carbon consumed
107.60 mg of TAG
342.30 mg of carbon consumed


In [32]:
# yield on glycerol

with lst:
    lst.reactions.EX_glc__D_e.lower_bound = 0
    lst.reactions.EX_glyc_e.lower_bound = -1
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f} growth on carbon source')
    lst.reactions.BIOMASS_Ls.lower_bound = percent*soln.objective_value
    lst.objective = 'SK_triglyc_LS_d'

    soln = lst.optimize()
    print(f'{soln.objective_value:.4f}')
    print(lst.objective)
    
    carbon_source_num_carbons = count_carbons_in_formula(lst.metabolites.glyc_e.formula)
    tag_num_carbons = count_carbons_in_formula(lst.metabolites.triglyc_LS_d.formula)    
    
    mgUptake = 1*lst.metabolites.glyc_e.formula_weight
    tag_produced = soln.objective_value*lst.metabolites.triglyc_LS_d.formula_weight
    
    print(f'the yield from 1 mmol of carbon is {tag_produced/mgUptake: .2f} g TAG/carbon consumed')
    print(f'the yield from 1 mmol of carbon is {soln.objective_value*tag_num_carbons/carbon_source_num_carbons: .2f} mmol-C TAG/mmol-C carbon consumed')

    print(f'{tag_produced:.2f} mg of TAG')
    print(f'{mgUptake:.2f} mg of carbon consumed')

0.0511 growth on carbon source
0.0004
Maximize
1.0*SK_triglyc_LS_d - 1.0*SK_triglyc_LS_d_reverse_50347
the yield from 1 mmol of carbon is  0.33 g TAG/carbon consumed
the yield from 1 mmol of carbon is  0.65 mmol-C TAG/mmol-C carbon consumed
30.60 mg of TAG
92.09 mg of carbon consumed


In [33]:
# yield on gluconate

with lst:
    lst.reactions.EX_glc__D_e.lower_bound = 0
    lst.reactions.EX_glcn_e.lower_bound = -1
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f} growth on carbon source')
    lst.reactions.BIOMASS_Ls.lower_bound = percent*soln.objective_value
    lst.objective = 'SK_triglyc_LS_d'

    soln = lst.optimize()
    print(f'{soln.objective_value:.4f}')
    print(lst.objective)
    
    carbon_source_num_carbons = count_carbons_in_formula(lst.metabolites.glcn_e.formula)
    tag_num_carbons = count_carbons_in_formula(lst.metabolites.triglyc_LS_d.formula)
    
    mgUptake = 1*lst.metabolites.glcn_e.formula_weight
    tag_produced = soln.objective_value*lst.metabolites.triglyc_LS_d.formula_weight
    
    print(f'the yield from 1 mmol of carbon is {tag_produced/mgUptake: .2f} g TAG/carbon consumed')
    print(f'the yield from 1 mmol of carbon is {soln.objective_value*tag_num_carbons/carbon_source_num_carbons: .2f} mmol-C TAG/mmol-C carbon consumed')
    
    print(f'{tag_produced:.2f} mg of TAG')
    print(f'{mgUptake:.2f} mg of carbon consumed')

0.0814 growth on carbon source
0.0006
Maximize
1.0*SK_triglyc_LS_d - 1.0*SK_triglyc_LS_d_reverse_50347
the yield from 1 mmol of carbon is  0.25 g TAG/carbon consumed
the yield from 1 mmol of carbon is  0.52 mmol-C TAG/mmol-C carbon consumed
48.99 mg of TAG
195.15 mg of carbon consumed


In [34]:
# yield on Hexadecanoate

with lst:
    lst.reactions.EX_glc__D_e.lower_bound = 0
    lst.reactions.EX_hdca_e.lower_bound = -1
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f} growth on carbon source')
    lst.reactions.BIOMASS_Ls.lower_bound = percent*soln.objective_value
    lst.objective = 'SK_triglyc_LS_d'

    soln = lst.optimize()
    print(f'{soln.objective_value:.4f}')
    print(lst.objective)
    
    carbon_source_num_carbons = count_carbons_in_formula(lst.metabolites.hdca_e.formula)
    tag_num_carbons = count_carbons_in_formula(lst.metabolites.triglyc_LS_d.formula)
    
    mgUptake = 1*lst.metabolites.hdca_e.formula_weight
    tag_produced = soln.objective_value*lst.metabolites.triglyc_LS_d.formula_weight
    
    print(f'the yield from 1 mmol of carbon is {tag_produced/mgUptake: .2f} g TAG/carbon consumed')
    print(f'the yield from 1 mmol of carbon is {soln.objective_value*tag_num_carbons/carbon_source_num_carbons: .2f} mmol-C TAG/mmol-C carbon consumed')
    
    print(f'{tag_produced:.2f} mg of TAG')
    print(f'{mgUptake:.2f} mg of carbon consumed')

0.2691 growth on carbon source
0.0025
Maximize
1.0*SK_triglyc_LS_d - 1.0*SK_triglyc_LS_d_reverse_50347
the yield from 1 mmol of carbon is  0.83 g TAG/carbon consumed
the yield from 1 mmol of carbon is  0.84 mmol-C TAG/mmol-C carbon consumed
211.04 mg of TAG
255.42 mg of carbon consumed


In [35]:
# yield on maltose.

with lst:
    lst.reactions.EX_glc__D_e.lower_bound = 0
    lst.reactions.EX_malt_e.lower_bound = -1
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f} growth on carbon source')
    lst.reactions.BIOMASS_Ls.lower_bound = percent*soln.objective_value
    lst.objective = 'SK_triglyc_LS_d'

    soln = lst.optimize()
    print(f'{soln.objective_value:.4f}')
    print(lst.objective)
    
    carbon_source_num_carbons = count_carbons_in_formula(lst.metabolites.malt_e.formula)
    tag_num_carbons = count_carbons_in_formula(lst.metabolites.triglyc_LS_d.formula)
    
    mgUptake = 1*lst.metabolites.malt_e.formula_weight
    tag_produced = soln.objective_value*lst.metabolites.triglyc_LS_d.formula_weight
    
    print(f'the yield from 1 mmol of carbon is {tag_produced/mgUptake: .2f} g TAG/carbon consumed')
    print(f'the yield from 1 mmol of carbon is {soln.objective_value*tag_num_carbons/carbon_source_num_carbons: .2f} mmol-C TAG/mmol-C carbon consumed')
    
    print(f'{tag_produced:.2f} mg of TAG')
    print(f'{mgUptake:.2f} mg of carbon consumed')

0.1786 growth on carbon source
0.0013
Maximize
1.0*SK_triglyc_LS_d - 1.0*SK_triglyc_LS_d_reverse_50347
the yield from 1 mmol of carbon is  0.31 g TAG/carbon consumed
the yield from 1 mmol of carbon is  0.57 mmol-C TAG/mmol-C carbon consumed
107.60 mg of TAG
342.30 mg of carbon consumed


In [36]:
# yield on acetate

with lst:
    lst.reactions.EX_glc__D_e.lower_bound = 0
    lst.reactions.EX_ac_e.lower_bound = -1
    soln = lst.optimize()
    print(f'{soln.objective_value:.4f} growth on carbon source')
    lst.reactions.BIOMASS_Ls.lower_bound = percent*soln.objective_value

    lst.objective = 'SK_triglyc_LS_d'

    soln = lst.optimize()
    print(f'{soln.objective_value:.4f}')
    print(lst.objective)
    
    carbon_source_num_carbons = count_carbons_in_formula(lst.metabolites.ac_e.formula)
    tag_num_carbons = count_carbons_in_formula(lst.metabolites.triglyc_LS_d.formula)
    
    mgUptake = 1*lst.metabolites.ac_e.formula_weight
    tag_produced = soln.objective_value*lst.metabolites.triglyc_LS_d.formula_weight
    
    print(f'the yield from 1 mmol of carbon is {tag_produced/mgUptake: .2f} g TAG/carbon consumed')
    print(f'the yield from 1 mmol of carbon is {soln.objective_value*tag_num_carbons/carbon_source_num_carbons: .2f} mmol-C TAG/mmol-C carbon consumed')
    
    print(f'{tag_produced:.2f} mg of TAG')
    print(f'{mgUptake:.2f} mg of carbon consumed')

0.0227 growth on carbon source
0.0002
Maximize
1.0*SK_triglyc_LS_d - 1.0*SK_triglyc_LS_d_reverse_50347
the yield from 1 mmol of carbon is  0.25 g TAG/carbon consumed
the yield from 1 mmol of carbon is  0.47 mmol-C TAG/mmol-C carbon consumed
14.57 mg of TAG
59.04 mg of carbon consumed


unblock the redox reactions. 

In [38]:
lst.reactions.ALCD2y.bounds=0,1000
lst.reactions.get_by_id('ALDD19x_P').bounds=0,1000
lst.reactions.HSDy.bounds = -1000,1000

In [39]:
soln = pfba(lst)
# soln.fluxes.to_csv('pfba_glc_for_fig_FORPUB_unblocked.csv')

In [40]:
lst

Name,Lst
Memory address,147f1a490
Number of metabolites,1909
Number of reactions,2193
Number of genes,996
Number of groups,0
Objective expression,1.0*BIOMASS_Ls - 1.0*BIOMASS_Ls_reverse_27d85
Compartments,"c, m, r, e, x, v, n, g, d"
